https://docs.databricks.com/aws/en/views/dynamic 

In [0]:
%sql
use catalog dev_catalog;
use schema dev_schema;

In [0]:
%sql

CREATE OR REPLACE TABLE heartrate_device (device_id INT, mrn STRING, name STRING, time TIMESTAMP, heartrate DOUBLE); 


INSERT INTO heartrate_device VALUES 
(23, "40580129", "Nicholas Spears", "2020-02-01T00:01:58.000+0000", 54.0122153343), 
(17, "52804177", "Lynn Russell", "2020-02-01T00:02:55.000+0000", 92.5136468131), 
(37, "65300842", "Samuel Hughes", "2020-02-01T00:08:58.000+0000", 52.1354807863), 
(23, "40580129", "Nicholas Spears", "2020-02-01T00:16:51.000+0000", 54.6477014191), 
(17, "52804177", "Lynn Russell", "2020-02-01T00:18:08.000+0000", 95.033344842); 

In [0]:
%sql
select * from heartrate_device;

In [0]:
%sql
CREATE OR REPLACE VIEW gold.agg_heartrate AS
 ( 
SELECT mrn, name, MEAN(heartrate) avg_heartrate, DATE_TRUNC("DD", time) date 
FROM heartrate_device 
GROUP BY mrn, name, DATE_TRUNC("DD", time)
); 


In [0]:
%sql
select * from gold.agg_heartrate;

## GRANT Permissions to users

In [0]:
GRANT USE CATALOG ON CATALOG dev_catalog TO `ap@pulianandyahoo.onmicrosoft.com`;

GRANT USE SCHEMA ON SCHEMA dev_schema.gold TO `ap@pulianandyahoo.onmicrosoft.com`;

GRANT SELECT ON TABLE dev_catalog.dev_schema.gold.agg_heartrate TO `pulianand@yahoo.com`;

### Column Level

In [0]:
%sql
GRANT USE CATALOG ON CATALOG dev_catalog TO `pulianand@yahoo.com`;

GRANT USE SCHEMA ON SCHEMA dev_schema.gold TO `pulianand@yahoo.com`;

GRANT SELECT ON TABLE dev_catalog.dev_schema.gold.agg_heartrate TO `pulianand@yahoo.com`;

### ROW Level

In [0]:
%sql
CREATE OR REPLACE VIEW gold.agg_heartrate AS 
SELECT 
mrn, 
time,
device_id, 
heartrate 
FROM heartrate_device 
WHERE 
CASE WHEN 
is_account_group_member('account users') THEN device_id < 30 
ELSE TRUE 
END 

### DAta masking Function

In [0]:
%sql
-- Data masking function 

CREATE OR REPLACE FUNCTION datamask(x STRING)
RETURNS STRING 
RETURN CONCAT(REPEAT("*", LENGTH(x) - 2), RIGHT(x, 2) 
); 


In [0]:
%sql
CREATE or replace VIEW gold.agg_heartrate AS 
SELECT 
CASE WHEN 
is_account_group_member('account users') THEN datamask(mrn) 
ELSE mrn 
END AS mrn, 
time, 
device_id, 
heartrate 
FROM heartrate_device 

In [0]:
%sql
select * from gold.agg_heartrate